In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

Mounted at /content/drive


In [ ]:
print("Start Anomaly Detection Model Data Preprocessing")

# reading data in
df = pd.read_csv("/content/drive/MyDrive/Data Mining Project/Notebooks/raw_data.csv")
df.drop(['Unnamed: 0'], axis=1, inplace = True)

Start Anomaly Detection Model Data Preprocessing


In [ ]:
# seems like null values are reading as strings. replace "None" with NoneType
df = df.apply(lambda col: col.replace("None", None, regex=True))

In [ ]:
# the measurements that are taken are at P1, P2, P5, P6, P8, and P10
# impute the rows where measurements aren't taken with the measurement prior
df_measurement_imputed = df.copy()
df_measurement_imputed['measurement'] = df_measurement_imputed.groupby('unit_id')['measurement'].fillna(method='ffill')
df_measurement_imputed['measurement'] = df_measurement_imputed['measurement'].astype(float)

In [ ]:
# changing unit_id and process_id  to numeric for ordering purposes
df_numeric = df_measurement_imputed.copy()
df_numeric['unit_id'] = df_numeric['unit_id'].str.replace("Unit ", '')
df_numeric['unit_id'] = pd.to_numeric(df_numeric['unit_id'])
df_numeric['process_id'] = df_numeric['process_id'].str.replace("P", '')
df_numeric['process_id'] = pd.to_numeric(df_numeric['process_id'])

In [ ]:
# calculating cycle time; the time between processes for each unit
df_ct = df_numeric.sort_values(by=['process_id','timestamp'])
df_ct['cycle_time'] = df_ct.groupby(['unit_id'])['timestamp'].diff()

# reconciling null values (first process for each unit)
# starting with setting first value cycle time as timestamp
df_ct.loc[(df_ct['unit_id'] == 1) & (df_ct['process_id']==1), "cycle_time"] = df_ct.loc[(df_ct['unit_id'] == 1) & (df_ct['process_id']==1), 'timestamp']
df_ct.loc[(df_ct['unit_id'] == 2) & (df_ct['process_id']==1), "cycle_time"] = df_ct.loc[(df_ct['unit_id'] == 2) & (df_ct['process_id']==1), 'timestamp']
df_ct.loc[(df_ct['unit_id'] == 3) & (df_ct['process_id']==1), "cycle_time"] = df_ct.loc[(df_ct['unit_id'] == 3) & (df_ct['process_id']==1), 'timestamp']

# using the previous instance of operator_id to find first process cycle time
null_rows = df_ct.sort_values(by=['operator_id', 'timestamp'])['cycle_time'].isnull()
df_ct.loc[null_rows, 'cycle_time'] = df_ct.groupby('operator_id')['timestamp'].diff()

In [ ]:
#Find first unit and last unit in the manufacturing line
min = df_ct['timestamp'].idxmin()
first_unit = df_ct.loc[min, 'unit_id']
max = df_ct['timestamp'].idxmax()
last_unit = df_ct.loc[max, 'unit_id']

#Trim the data to remove the ramp-up and ramp-down times during manufacturing
df_trimmed = df_ct[(df_ct['unit_id'] != first_unit) & (df_ct['unit_id'] != last_unit)]

In [ ]:
# adding average measurement at that process feature
df_with_average = df_trimmed.copy()
df_with_average['process_avg_measurement'] = df_with_average.groupby('process_id')['measurement'].transform('mean')

# adding average cycle time at that process
df_with_average['process_avg_cycletime'] = df_with_average.groupby('process_id')['cycle_time'].transform('mean')

# adding absolute difference between average and measurement/cycletime
df_with_average['dist_from_measure_avg'] = abs(df_with_average['process_avg_measurement']-df_with_average['measurement'])
df_with_average['dist_from_ct_avg'] = abs(df_with_average['process_avg_cycletime']-df_with_average['cycle_time'])

In [ ]:
# OHE for the unit_id, process_id, and operator_id
df_encoded = pd.get_dummies(df_with_average, columns=['operator_id', 'process_id'], drop_first=True)

# Concatenate the original DataFrame with the dummy columns
df_encoded = pd.concat([df_encoded, df_with_average[['process_id','operator_id']]], axis=1)

In [ ]:
# Adding lag features
num_lags = 3

df_lag = pd.DataFrame()

# columns to create lag features on
cols_to_lag = ['measurement']

# iterate through each unit
for unit in df_encoded['unit_id'].unique():
  df_unit = df_encoded[df_encoded['unit_id'] == unit]
  df_unit = df_unit.sort_values('timestamp')

  # create lag columns for each feature to lag
  for col in cols_to_lag:
    for i in range(1, num_lags + 1):
        df_unit[f'{col}_lag_{i}'] = df_unit[col].shift(i)

  # union of unit level df to total df
  df_lag = pd.concat([df_lag, df_unit], axis=0, ignore_index = True)

df_lag.fillna(0, inplace=True)

In [ ]:
# scaling data :)
scaler = MinMaxScaler()
df_scaled = df_lag[['timestamp', 'measurement',
       'cycle_time', 'process_avg_measurement', 'process_avg_cycletime',
       'dist_from_measure_avg', 'dist_from_ct_avg', 'operator_id_P1-2',
       'operator_id_P1-3', 'operator_id_P10-1', 'operator_id_P10-2',
       'operator_id_P10-3', 'operator_id_P10-4', 'operator_id_P10-5',
       'operator_id_P10-6', 'operator_id_P2-1', 'operator_id_P2-2',
       'operator_id_P2-3', 'operator_id_P3-1', 'operator_id_P3-2',
       'operator_id_P3-3', 'operator_id_P3-4', 'operator_id_P3-5',
       'operator_id_P4-1', 'operator_id_P4-2', 'operator_id_P4-3',
       'operator_id_P5-1', 'operator_id_P5-2', 'operator_id_P5-3',
       'operator_id_P5-4', 'operator_id_P5-5', 'operator_id_P6-1',
       'operator_id_P6-2', 'operator_id_P6-3', 'operator_id_P6-4',
       'operator_id_P6-5', 'operator_id_P6-6', 'operator_id_P7-1',
       'operator_id_P7-2', 'operator_id_P7-3', 'operator_id_P7-4',
       'operator_id_P7-5', 'operator_id_P8-1', 'operator_id_P8-2',
       'operator_id_P9-1', 'operator_id_P9-2', 'operator_id_P9-3',
       'operator_id_P9-4', 'operator_id_P9-5', 'operator_id_P9-6',
       'process_id_2', 'process_id_3', 'process_id_4', 'process_id_5',
       'process_id_6', 'process_id_7', 'process_id_8', 'process_id_9',
       'process_id_10', 'measurement_lag_1',
       'measurement_lag_2', 'measurement_lag_3']]
df_non_numeric = df_lag[list(set(df_lag.columns.tolist()) - set(df_scaled.columns.tolist()))]
df_scaled = scaler.fit_transform(df_scaled.to_numpy())
df_scaled = pd.DataFrame(df_scaled,columns=['timestamp', 'measurement',
       'cycle_time', 'process_avg_measurement', 'process_avg_cycletime',
       'dist_from_measure_avg', 'dist_from_ct_avg', 'operator_id_P1-2',
       'operator_id_P1-3', 'operator_id_P10-1', 'operator_id_P10-2',
       'operator_id_P10-3', 'operator_id_P10-4', 'operator_id_P10-5',
       'operator_id_P10-6', 'operator_id_P2-1', 'operator_id_P2-2',
       'operator_id_P2-3', 'operator_id_P3-1', 'operator_id_P3-2',
       'operator_id_P3-3', 'operator_id_P3-4', 'operator_id_P3-5',
       'operator_id_P4-1', 'operator_id_P4-2', 'operator_id_P4-3',
       'operator_id_P5-1', 'operator_id_P5-2', 'operator_id_P5-3',
       'operator_id_P5-4', 'operator_id_P5-5', 'operator_id_P6-1',
       'operator_id_P6-2', 'operator_id_P6-3', 'operator_id_P6-4',
       'operator_id_P6-5', 'operator_id_P6-6', 'operator_id_P7-1',
       'operator_id_P7-2', 'operator_id_P7-3', 'operator_id_P7-4',
       'operator_id_P7-5', 'operator_id_P8-1', 'operator_id_P8-2',
       'operator_id_P9-1', 'operator_id_P9-2', 'operator_id_P9-3',
       'operator_id_P9-4', 'operator_id_P9-5', 'operator_id_P9-6',
       'process_id_2', 'process_id_3', 'process_id_4', 'process_id_5',
       'process_id_6', 'process_id_7', 'process_id_8', 'process_id_9',
       'process_id_10', 'measurement_lag_1',
       'measurement_lag_2', 'measurement_lag_3'])

df_final = pd.concat([df_scaled, df_non_numeric], axis=1)

In [ ]:
df_final.to_csv("/content/drive/MyDrive/Data Mining Project/Notebooks/anomaly_detection_data.csv")

In [ ]:
df_final.sort_values('timestamp').head(100)

,timestamp,measurement,cycle_time,process_avg_measurement,process_avg_cycletime,dist_from_measure_avg,dist_from_ct_avg,operator_id_P1-2,operator_id_P1-3,operator_id_P10-1,...,process_id_7,process_id_8,process_id_9,process_id_10,measurement_lag_1,measurement_lag_2,measurement_lag_3,process_id,operator_id,unit_id
0,0.000000,0.011195,0.007265,0.000000,0.002588,0.513970,0.003978,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1,P1-2,2
8,0.001015,0.000000,0.013233,0.000000,0.002588,0.624533,0.015550,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1,P1-3,3
18,0.001677,0.007201,0.006227,0.000000,0.002588,0.099963,0.001965,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1,P1-1,4
26,0.002251,0.011195,0.004411,0.000000,0.002588,0.513970,0.001509,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1,P1-2,5
1,0.002516,0.098645,0.005968,0.097047,0.052867,0.324925,0.045319,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.096039,0.000000,0.000000,2,P2-2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.020389,0.303882,0.012974,0.301448,0.016141,0.022333,0.002423,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.175985,0.175985,0.385619,5,P5-1,11
14,0.020433,0.389469,0.009341,0.398401,0.012025,1.000000,0.000740,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.441856,0.363936,0.388037,7,P7-2,3
100,0.020610,0.304236,0.010119,0.301448,0.016141,0.059031,0.003064,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.183811,0.183811,0.402767,5,P5-3,13
238,0.020610,0.007201,0.005189,0.000000,0.002588,0.099963,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1,P1-1,28
